In [1]:
val spark = SparkSession.builder().appName("lab05").getOrCreate()
import spark.implicits._
val sc = spark.sparkContext
sc.applicationId

spark = org.apache.spark.sql.SparkSession@692d34f1
sc = org.apache.spark.SparkContext@7b0fefae


application_1667306389915_1620

In [2]:
import scala.sys.process._
import org.apache.spark.sql.functions.{udf, lit, col}
import org.apache.spark.sql.types.{IntegerType, DoubleType, FloatType, ArrayType}
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.linalg.DenseVector
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.classification.GBTClassifier
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

def toArray: DenseVector => Array[Double] = _.toArray
def toArrayUdf = udf(toArray).asNonNullable()

toArray = > Array[Double]


toArrayUdf: org.apache.spark.sql.expressions.UserDefinedFunction


> Array[Double]

In [3]:
val options: Map[String, String] = Map("header" -> "true", "inferSchema" -> "true")
val df_train = spark.read.options(options).csv("/labs/slaba05/lab05_train.csv").drop("_c0")
df_train.count

options = Map(header -> true, inferSchema -> true)
df_train = [ID: int, CR_PROD_CNT_IL: int ... 114 more fields]


320764

In [4]:
val num_col_names = df_train.dtypes.filter(x => Seq("IntegerType", "DoubleType").contains(x._2.toString)).map(_._1)
val df_train_numeric = df_train.select(num_col_names.map(col): _*).na.fill(0)

num_col_names = Array(ID, CR_PROD_CNT_IL, AMOUNT_RUB_CLO_PRC, PRC_ACCEPTS_A_EMAIL_LINK, APP_REGISTR_RGN_CODE, PRC_ACCEPTS_A_POS, PRC_ACCEPTS_A_TK, TURNOVER_DYNAMIC_IL_1M, CNT_TRAN_AUT_TENDENCY1M, SUM_TRAN_AUT_TENDENCY1M, AMOUNT_RUB_SUP_PRC, PRC_ACCEPTS_A_AMOBILE, SUM_TRAN_AUT_TENDENCY3M, PRC_ACCEPTS_TK, PRC_ACCEPTS_A_MTP, REST_DYNAMIC_FDEP_1M, CNT_TRAN_AUT_TENDENCY3M, CNT_ACCEPTS_TK, REST_DYNAMIC_SAVE_3M, CR_PROD_CNT_VCU, REST_AVG_CUR, CNT_TRAN_MED_TENDENCY1M, AMOUNT_RUB_NAS_PRC, TRANS_COUNT_SUP_PRC, CNT_TRAN_CLO_TENDENCY1M, SUM_TRAN_MED_TENDENCY1M, PRC_ACCEPTS_A_ATM, PRC_ACCEPTS_MTP, TRANS_COUNT_NAS_PRC, CNT_ACCEPTS_MTP, CR_PROD_CNT_TOVR, CR_PROD_CNT_PIL, SUM_TRAN_CLO_TENDENCY1M, TURNOVER_CC, TRANS_COUNT_ATM_PRC, AMOUNT_RUB_ATM_PRC, TURNOVER_PAYM, AGE, CNT_TRAN_MED_TENDE...


Array(ID, CR_PROD_CNT_IL, AMOUNT_RUB_CLO_PRC, PRC_ACCEPTS_A_EMAIL_LINK, APP_REGISTR_RGN_CODE, PRC_ACCEPTS_A_POS, PRC_ACCEPTS_A_TK, TURNOVER_DYNAMIC_IL_1M, CNT_TRAN_AUT_TENDENCY1M, SUM_TRAN_AUT_TENDENCY1M, AMOUNT_RUB_SUP_PRC, PRC_ACCEPTS_A_AMOBILE, SUM_TRAN_AUT_TENDENCY3M, PRC_ACCEPTS_TK, PRC_ACCEPTS_A_MTP, REST_DYNAMIC_FDEP_1M, CNT_TRAN_AUT_TENDENCY3M, CNT_ACCEPTS_TK, REST_DYNAMIC_SAVE_3M, CR_PROD_CNT_VCU, REST_AVG_CUR, CNT_TRAN_MED_TENDENCY1M, AMOUNT_RUB_NAS_PRC, TRANS_COUNT_SUP_PRC, CNT_TRAN_CLO_TENDENCY1M, SUM_TRAN_MED_TENDENCY1M, PRC_ACCEPTS_A_ATM, PRC_ACCEPTS_MTP, TRANS_COUNT_NAS_PRC, CNT_ACCEPTS_MTP, CR_PROD_CNT_TOVR, CR_PROD_CNT_PIL, SUM_TRAN_CLO_TENDENCY1M, TURNOVER_CC, TRANS_COUNT_ATM_PRC, AMOUNT_RUB_ATM_PRC, TURNOVER_PAYM, AGE, CNT_TRAN_MED_TENDE...

### Анализ корреляции

In [5]:
val assembler = new VectorAssembler().setInputCols(num_col_names).setOutputCol("features")
val df_train_vector = assembler.transform(df_train_numeric).select("features")
val matrix = Correlation.corr(df_train_vector, "features").collect()
val dm = matrix.head.getAs[org.apache.spark.ml.linalg.DenseMatrix](0)
var target_corr_values_tmp: org.apache.spark.ml.linalg.Vector = null
for (i <- dm.colIter) target_corr_values_tmp = i // Получение последнего столбца матрицы
val target_corr_values = target_corr_values_tmp.toArray

assembler = vecAssembler_2547e51e6df4
df_train_vector = [features: vector]
matrix = 


Array([1.0                     3.375606502089509E-4    ... (103 total)
3.375606502089509E-4    1.0                     ...
0.004106393073545081    -0.020620213921799656   ...
NaN                     NaN                     ...
-4.238074492499304E-4   0.3802756819726169      ...
NaN                     NaN                     ...
NaN                     NaN                     ...
9.339625116887346E-4    0.13312023493983147     ...
0.01828957955423523     0.01407161057115469     ...
0.016404585162782783    0.01386474268384372     ...
0.012625233873670955    -0.004545546581042047   ...
NaN         ...


In [6]:
val target_corr = num_col_names.zip(target_corr_values)
val target_corr_not_nan = target_corr.filter(x => !x._2.isNaN)
val cols = target_corr_not_nan.map(_._1).filter(x => !Seq("ID", "TARGET").contains(x)).toArray

target_corr = Array((ID,0.003387446612468513), (CR_PROD_CNT_IL,0.012684655349955555), (AMOUNT_RUB_CLO_PRC,-0.03782165304131623), (PRC_ACCEPTS_A_EMAIL_LINK,NaN), (APP_REGISTR_RGN_CODE,0.020482417428626935), (PRC_ACCEPTS_A_POS,NaN), (PRC_ACCEPTS_A_TK,NaN), (TURNOVER_DYNAMIC_IL_1M,0.02159975577580789), (CNT_TRAN_AUT_TENDENCY1M,-0.019067662697206042), (SUM_TRAN_AUT_TENDENCY1M,-0.018532081956130295), (AMOUNT_RUB_SUP_PRC,-0.017040956746899347), (PRC_ACCEPTS_A_AMOBILE,NaN), (SUM_TRAN_AUT_TENDENCY3M,-0.03525372214621112), (PRC_ACCEPTS_TK,NaN), (PRC_ACCEPTS_A_MTP,NaN), (REST_DYNAMIC_FDEP_1M,0.021244579613353975), (CNT_TRAN_AUT_TENDENCY3M,-0.035829341366110316), (CNT_ACCEPTS_TK,NaN), (REST_DYNAMIC_SAVE_3M,0.015354591850949744), (CR_PROD_CNT_VCU,0.004849403104617076), (RES...


Array((ID,0.003387446612468513), (CR_PROD_CNT_IL,0.012684655349955555), (AMOUNT_RUB_CLO_PRC,-0.03782165304131623), (PRC_ACCEPTS_A_EMAIL_LINK,NaN), (APP_REGISTR_RGN_CODE,0.020482417428626935), (PRC_ACCEPTS_A_POS,NaN), (PRC_ACCEPTS_A_TK,NaN), (TURNOVER_DYNAMIC_IL_1M,0.02159975577580789), (CNT_TRAN_AUT_TENDENCY1M,-0.019067662697206042), (SUM_TRAN_AUT_TENDENCY1M,-0.018532081956130295), (AMOUNT_RUB_SUP_PRC,-0.017040956746899347), (PRC_ACCEPTS_A_AMOBILE,NaN), (SUM_TRAN_AUT_TENDENCY3M,-0.03525372214621112), (PRC_ACCEPTS_TK,NaN), (PRC_ACCEPTS_A_MTP,NaN), (REST_DYNAMIC_FDEP_1M,0.021244579613353975), (CNT_TRAN_AUT_TENDENCY3M,-0.035829341366110316), (CNT_ACCEPTS_TK,NaN), (REST_DYNAMIC_SAVE_3M,0.015354591850949744), (CR_PROD_CNT_VCU,0.004849403104617076), (RES...

### Обучение модели

In [7]:
val assembler = new VectorAssembler().setInputCols(cols).setOutputCol("features")
val gbtc = new GBTClassifier().setLabelCol("TARGET").setMaxIter(10)
val pipeline = new Pipeline().setStages(Array(assembler, gbtc))
val pipeline_model = pipeline.fit(df_train_numeric)

assembler = vecAssembler_3ddb6770fa1b
gbtc = gbtc_7e17cce285c4
pipeline = pipeline_1c1f888e0443
pipeline_model = pipeline_1c1f888e0443


pipeline_1c1f888e0443

### Валидация модели

In [8]:
val train_predict = pipeline_model.transform(df_train_numeric)
train_predict.select("id", "features", "rawPrediction", "probability", "prediction").show(10, 30)

+------+------------------------------+------------------------------+------------------------------+----------+
|    id|                      features|                 rawPrediction|                   probability|prediction|
+------+------------------------------+------------------------------+------------------------------+----------+
|479990|(91,[12,19,23,24,26,40,45,4...|[1.250236441855763,-1.25023...|[0.9241749642360747,0.07582...|       0.0|
|450480|(91,[6,12,14,15,18,19,23,24...|[1.1168899880930325,-1.1168...|[0.90324221664835,0.0967577...|       0.0|
|473954|(91,[6,12,13,14,15,17,18,19...|[0.7768622574925201,-0.7768...|[0.825451016694544,0.174548...|       0.0|
|531038|(91,[12,23,24,26,40,45,67],...|[1.2957717911171909,-1.2957...|[0.9303153520420651,0.06968...|       0.0|
|349303|(91,[6,7,9,12,15,23,24,26,4...|[1.2432709397212303,-1.2432...|[0.9231929537591829,0.07680...|       0.0|
|200239|(91,[12,26,40,45,47,64,67],...|[1.2957717911171909,-1.2957...|[0.9303153520420651,0.0696

train_predict = [ID: int, CR_PROD_CNT_IL: int ... 105 more fields]


[ID: int, CR_PROD_CNT_IL: int ... 105 more fields]

In [9]:
val evaluator = new BinaryClassificationEvaluator()
.setRawPredictionCol("rawPrediction")
.setLabelCol("prediction")
.setMetricName("areaUnderROC")
evaluator.evaluate(train_predict)

evaluator = binEval_caf9657e96c3


1.0

### Предсказание на тестовых данных

In [10]:
val options: Map[String, String] = Map("header" -> "true", "inferSchema" -> "true")
val df_test = spark.read.options(options).csv("/labs/slaba05/lab05_test.csv").withColumn("target", lit(0))
val df_test_numeric = df_test.select(num_col_names.map(col): _*).na.fill(0)
val test_predict = pipeline_model.transform(df_test_numeric)
test_predict.select("id", "features", "rawPrediction", "probability", "prediction").show(10, 30)

+------+------------------------------+------------------------------+------------------------------+----------+
|    id|                      features|                 rawPrediction|                   probability|prediction|
+------+------------------------------+------------------------------+------------------------------+----------+
|519130|(91,[10,12,23,24,26,40,45,4...|[0.7957731030558924,-0.7957...|[0.8308335322374557,0.16916...|       0.0|
|234045|(91,[1,6,12,14,15,16,18,21,...|[1.2702109722902675,-1.2702...|[0.9269274112925896,0.07307...|       0.0|
|401256|(91,[1,4,5,6,7,9,12,14,15,1...|[1.2852327943616637,-1.2852...|[0.928936437210913,0.071063...|       0.0|
|551070|(91,[23,24,25,26,41,48,51,5...|[1.1465928520397248,-1.1465...|[0.9083111073336182,0.09168...|       0.0|
|367285|(91,[12,26,40,45,67],[14842...|[1.2848890638425159,-1.2848...|[0.9288910420948985,0.07110...|       0.0|
|497998|(91,[6,12,14,15,18,19,23,24...|[1.2797236618674144,-1.2797...|[0.9282056361419719,0.0717

options = Map(header -> true, inferSchema -> true)
df_test = [_c0: int, ID: int ... 115 more fields]
df_test_numeric = [ID: int, CR_PROD_CNT_IL: int ... 101 more fields]
test_predict = [ID: int, CR_PROD_CNT_IL: int ... 105 more fields]


[ID: int, CR_PROD_CNT_IL: int ... 105 more fields]

In [11]:
val options: Map[String, String] = Map("header" -> "True", "delimiter" -> "\t")
val test_target = test_predict.withColumn("target", toArrayUdf(col("probability")).getItem(1).cast(FloatType)).select("id", "target")
test_target.coalesce(1).write.mode("overwrite").options(options).csv("tmp/lab05")

options = Map(header -> True, delimiter -> "	")
test_target = [id: int, target: float]


[id: int, target: float]

In [12]:
"rm -f lab05.csv".!!
"hdfs dfs -get tmp/lab05/part-* lab05.csv".!!

res56: String = ""


In [13]:
sc.stop